### LIBRARIES :

In [1]:
import pandas as pd

### READ THE DATASET :

In [2]:
df = pd.read_csv('bank-full.csv', delimiter=';', quotechar='"')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


### Question 1
Install Pipenv

What's the version of pipenv you installed?

Use --version to find out

In [5]:
!pip install pipenv

In [7]:
!pipenv --version

pipenv, version 2024.2.0


### Question 2
Use Pipenv to install Scikit-Learn version 1.5.2

What's the first hash for scikit-learn you get in Pipfile.lock?

In [9]:
!pipenv install scikit-learn==1.5.2

To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing scikit-learn==1.5.2...
Installation Succeeded
Building requirements...
[    ] Locking packages...
Resolving dependencies...
[    ] Locking packages...
[=   ] Locking packages...
[==  ] Locking packages...
[=== ] Locking packages...
[ ===] Locking packages...
[  ==] Locking packages...
[   =] Locking packages...
[    ] Locking packages...
[   =] Locking packages...
[  ==] Locking packages...
[ ===] Locking packages...
[=== ] Locking packages...
[==  ] Locking packages...
[=   ] Locking packages...
[    ] Locking packages...
[=   ] Locking packages...
[==  ] Locking packages...
[=== ] Locking packages...
[ ===] Locking packages...
[  ==] Locking packages...
[   =] Locking packages...
[    ] Locking packages...
[   =] Locking packages...
[  ==] Locking packages...
[ ===] Locking packages...
[====] Locking packages...
[=== ] Locking packages...
[=   ] Lock

Creating a Pipfile for this project...
Pipfile.lock not found, creating...
Locking [packages] dependencies...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (702ad05de9bc9de99a4807c8dde1686f31e0041d7b5f6f6b74861195a52110f5)!


### This is the hash for scikit-learn from Pipfile.lock

### 03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445

### Models
We've prepared a dictionary vectorizer and a model.

They were trained (roughly) using this code:

features = ['job', 'duration', 'poutcome']

dicts = df[features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)

X = dv.fit_transform(dicts)

model = LogisticRegression().fit(X, y)

Note: You don't need to train the model. This code is just for your reference.

And then saved with Pickle. Download them:

DictVectorizer
LogisticRegression

With wget:

PREFIX=https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework

wget $PREFIX/model1.bin

wget $PREFIX/dv.bin

### Question 3
Let's use these models!

Write a script for loading these models with pickle
Score this client:

{"job": "management", "duration": 400, "poutcome": "success"}

What's the probability that this client will get a subscription?

0.359

0.559

0.759

0.959

In [13]:
import pickle
import warnings

warnings.filterwarnings("ignore")

with open('dv.bin', 'rb') as dv_file:
    dv = pickle.load(dv_file)

with open('model1.bin', 'rb') as model_file:
    model = pickle.load(model_file)

client = {"job": "management", "duration": 400, "poutcome": "success"}

X = dv.transform([client])

prob = model.predict_proba(X)[0, 1]
print(f"Subscription probability: {prob:.3f}")

Subscription probability: 0.759


In [15]:
import hashlib

def get_md5(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

model1_md5 = get_md5('model1.bin')
dv_md5 = get_md5('dv.bin')

print(f"MD5 for model1.bin: {model1_md5}")
print(f"MD5 for dv.bin: {dv_md5}")

MD5 for model1.bin: 3d8bb28974e55edefa000fe38fd3ed12
MD5 for dv.bin: 7d37616e00aa80f2152b8b0511fc2dff


### Question 4
Now let's serve this model as a web service

Install Flask and gunicorn (or waitress, if you're on Windows)
Write Flask code for serving the model
Now score this client using requests:

url = "YOUR_URL"

client = {"job": "student", "duration": 280, "poutcome": "failure"}

requests.post(url, json=client).json()

What's the probability that this client will get a subscription?

0.335

0.535

0.735

0.935

In [17]:
!pip install Flask waitress

In [19]:
dicts: list[dict] = [{"job": "student", "duration": 280, "poutcome": "failure"}]
y_predict = model.predict_proba(X=dv.transform(X=dicts))[:, 1]
round(number=y_predict[0], ndigits=3)

0.335

#### Flask app

In [7]:
record: dict[str, int | str] = {"job": "student", "duration": 280, "poutcome": "failure"}

url = "http://127.0.0.1:8888/predict"

In [16]:
import requests
headers = {
    "Content-Type": "application/json",
}
response = requests.post(url=url, json=record, headers=headers)

In [ ]:
content = response.json()
print(round(number=content["probability"], ndigits=3))

### Question 5
Download the base image svizor/zoomcamp-model:3.11.5-slim. You can easily make it by using docker pull command.

So what's the size of this base image?

45 MB

130 MB

245 MB

330 MB

You can get this information when running docker images - it'll be in the "SIZE" column.

### In docker in size column, for image svizor/zoomcamp-model:3.11.5-slim - 130MB

### Question 6
Let's run your docker container!

After running it, score this client once again:

url = "YOUR_URL"
client = {"job": "management", "duration": 400, "poutcome": "success"}
requests.post(url, json=client).json()
What's the probability that this client will get a subscription now?

0.287

0.530

0.757

0.960

In [8]:
record: dict[str, int | str] = {"job": "management", "duration": 400, "poutcome": "success"}

url = "http://127.0.0.1:8888/predict"

In [14]:
import requests
headers = {
    "Content-Type": "application/json",
}
response = requests.post(url=url, json=record, headers=headers)

In [ ]:
content = response.json()
print(round(number=content["probability"], ndigits=3))

### Answer is : 0.757 